Import PyTorch libraries

In [2]:
import os
import torch
from torch import nn
torch.__version__
import matplotlib.pyplot as plt


### Check Our Processing Capability (CPU vs. GPU)<br>
When developing A.I. projects, it will help to have a powerful GPU.  While this project does not require one, the code below will detect if one is present in your environment and use it during the training process.<br>


Check to see if we have a GPU to use for training

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was detected.'.format(device))
#Print the name of the cuda device, if detected
if device=='cuda':
    print (torch.torch.cuda.get_device_name(device=device))

A cpu device was detected.



## Step 2 - Download and Prepare our Dataset<br>
When training a neural network from scratch, you will usually need a lot of data.  We will start by loading all the lemonade stand data for one year (365 items) which is a rather small, simply synthetic dataset.  It includes information about the day the lemonade was sold including whether or not it was a weekend, sunny, warm, a big sign was present to advertise and the price.  Finally, there is the number of lemonade's sold.  Our neural network will be trained to predict the number of lemonade's sold (output) based on the other attributes (inputs).<br>


Use Pandas to do our dataprocessing on the dataset<br>
Download the dataset

In [4]:
import pandas as pd
url = 'https://raw.githubusercontent.com/federicoq1997/NeuralNetwork/main/Esportazione_Articoli_1106_20221127_153540.csv'
df = pd.read_csv(url)
df.head(10)

,qty,price,day1,day2,day3,day4,day5,day6,day7,date
0,1,1.0,1,0,0,0,0,0,0,2021-05-31
1,2,1.0,0,0,1,0,0,0,0,2021-06-02
2,1,1.0,0,0,1,0,0,0,0,2021-06-02
3,1,1.0,0,0,0,1,0,0,0,2021-06-03
4,1,1.0,0,0,0,0,1,0,0,2021-06-18
5,2,1.0,0,0,0,0,1,0,0,2021-06-18
6,1,1.0,0,0,0,0,1,0,0,2021-06-25
7,1,1.0,0,1,0,0,0,0,0,2021-07-06
8,1,1.0,1,0,0,0,0,0,0,2021-07-12
9,1,1.0,1,0,0,0,0,0,0,2021-07-12


Check the size/shape of our dataset

In [5]:
df.shape

(29764, 10)


### Create our Inputs and Outputs for Training our Neural Network<br>
The data has been collected in a table with the following columns:  <br>
<pre> Weekend Sunny Warm BigSign price qty</pre><br>
While the dataset is more or less ready to be used, we have two fields (price and qty) that contain real values.  Usually, it's easier to train neural networks if the values used are in the range rough range of -1..1.  We will first reduce the range of price and qty down using standardization. <br>


Calculate the mean and standard deviation of price<br>
Standardize numSold

In [6]:
priceMean = df['price'].mean()
priceStd = df['price'].std()
df['price'] = (df['price']-priceMean)/priceStd

Calculate the mean and standard deviation of numSold<br>
Standardize numSold

In [7]:
numSoldMean = df['qty'].mean()
numSoldStd = df['qty'].std()
df['qty'] = (df['qty']-numSoldMean)/numSoldStd

In [8]:
df.head()

,qty,price,day1,day2,day3,day4,day5,day6,day7,date
0,-0.659074,-1.027964,1,0,0,0,0,0,0,2021-05-31
1,0.474969,-1.027964,0,0,1,0,0,0,0,2021-06-02
2,-0.659074,-1.027964,0,0,1,0,0,0,0,2021-06-02
3,-0.659074,-1.027964,0,0,0,1,0,0,0,2021-06-03
4,-0.659074,-1.027964,0,0,0,0,1,0,0,2021-06-18



### Create our Input (x) and Ouput (y) to Train our Neural Network<br>
 <br>
Here you will create the input (x) and output (y) variables needed to train our network.  The number we want our neural network to predict is the field called 'qty'.  This will be the output (y).  We will need to seperate out our input (Weekend, Sunny, Warm, BigSign, price) from the ouput (qty).<br>


Create our PyTorch tensors and move to CPU or GPU if available<br>
Extract the inputs and create a PyTorch tensor x (inputs)

In [9]:
inputs = ['day1','day2','day3','day4','day5','day6','day7','price']
x = torch.tensor(df[inputs].values, dtype=torch.float, device=device)

Extract the outputs and create a PyTorch tensor y (outputs)

In [10]:
outputs = ['qty']
y = torch.tensor(df[outputs].values,dtype=torch.float, device=device)

Explore the first 5 inputs

In [11]:
x[:]

tensor([[ 1.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -1.0280],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000, -1.0280],
        [ 0.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000, -1.0280],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  0.0000,  0.3977],
        [ 0.0000,  0.0000,  0.0000,  ...,  1.0000,  0.0000,  0.3977],
        [ 0.0000,  1.0000,  0.0000,  ...,  0.0000,  0.0000,  0.3977]])

Explore the first 5 outputs

In [12]:
y[:5]

tensor([[-0.6591],
        [ 0.4750],
        [-0.6591],
        [-0.6591],
        [-0.6591]])

In [35]:
for i in y:
	if not torch.isfinite(i).all():
		print(i)


## Step 3 - Build your Neural Network<br>
Below you will build a simply neural network that will take in the inputs above (5) and produce a single value as an output.  This network has a single hidden layer of 100 units.<br>


Define your PyTorch neural network<br>
Number of Inputs: 8<br>
Number of Hidden Units: 100<br>
Number of Hidden Layers: 1<br>
Activation Function:  Relu<br>
Number of Ouputs: 1

In [16]:
model = nn.Sequential(
            nn.Linear(8,100),
            nn.ReLU(),
            nn.Linear(100,1)
        )

Move it to either the CPU or GPU depending on what we have available

In [17]:
model.to(device)

Sequential(
  (0): Linear(in_features=8, out_features=160, bias=True)
  (1): ReLU()
  (2): Linear(in_features=160, out_features=1, bias=True)
)


## Step 4 - Train your Neural Network<br>
Here we will simply train our neural network on the dataset.  We will provide it with an input and and output.  The training loop will then adjust the weights within the neural network to make it more accuarate as we go through the training process.<br>


In [19]:
import torch.optim as optim
#Meausure our neural network by mean square error
criterion = torch.nn.MSELoss()
#Train our network with a simple SGD approach
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9)
for epoch in range(8):
    totalLoss = 0
    for i in range(len(x)):
        print('x[i]',x[i])
        # Single Forward Pass
        ypred = model(x[i])
        print('ypred',ypred)
        # Measure how well the model predicted vs actual
        loss = criterion(ypred,y[i])
        #print('loss',loss)
        # Track how well the model predicted
        totalLoss+=loss.item()
        
        # Update the neural network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print out our loss after each training iteration
    print ("Total Loss: ", totalLoss)

x[i] tensor([ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000, -1.0280])
ypred tensor([nan], grad_fn=<AddBackward0>)
x[i] tensor([ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.0280])
ypre

KeyboardInterrupt: 


## Step 5 - Analyze the Network's Performance

## Step 6 - Test with Your Own Predictions

In [ ]:
# Plot predictions vs. true values<br>
@torch.no_grad() 
def graphPredictions(model, x, y , minValue, maxValue):

    model.eval()                               # Set the model to inference mode<br>
  
    predictions=[]                             # Track predictions<br>
    actual=[]                                  # Track the actual labels<br>

    x.to(device)
    y.to(device)
    model.to(device)
    for i in range(len(x)):
        # Single forward pass<br>
        pred = model(x[i])
        # Un-normalize our prediction<br>
        pred = pred*numSoldStd+numSoldMean
        act = y[i]*numSoldStd+numSoldMean
    
        # Save prediction and actual label<br>
        predictions.append(pred.tolist())
        actual.append(act.item())
    
    # Plot actuals vs predictions<br>
    plt.scatter(actual, predictions)
    plt.xlabel('Actual Item Sold')
    plt.ylabel('Predicted Item Sold')
    plt.plot([minValue,maxValue], [minValue,maxValue]) 
    plt.xlim(minValue, maxValue)
    plt.ylim(minValue, maxValue)

    # Make the display equal in both dimensions<br>
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()
graphPredictions(model, x, y, 0, 300)

Below we use the synthetic data generator forumla to<br>
determine what the actual result should have been.

Data that affects the number of lemons sold in one day

In [ ]:
day1 = 1
day2 = 0
day3 = 0
day4 = 0
day5 = 0
day6 = 0
day7 = 0
price = 10
model.to('cpu')
price = (price - priceMean)/priceStd
x1 = torch.tensor([day1, day2, day3, day4, day5, day6, day7, price],dtype=float)
y1 = model(x1.float())
y1 = y1*numSoldStd+numSoldMean
   
# Compare what your network predicted to the actual
print ("Neural Network Predicts: ", y1.item())